### Pergunta 3
3.	Explique como você faria a previsão do preço a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('teste_indicium_precificacao.csv')
df['reviews_por_mes'] = df['reviews_por_mes'].replace(to_replace=np.nan, value=0).astype('int64')
df = df.drop(['nome', 'host_name', 'ultima_review', 'id', 'host_id'], axis=1)
# Implementando IQR

# Obtendo o prmeito quartil e o terceiro quartil
q1 = df["price"].quantile(0.25)
q3 = df["price"].quantile(0.75)

# Calculando o IQR
iqr = q3 - q1

#Definindo limites para outliers
lim_inf = q1 - (1.5*iqr)
lim_sup = q3 + (1.5*iqr)

# Filtando os dados
df_novo = df[(df["price"] >= lim_inf) & (df["price"] <= lim_sup)]
df_novo = df_novo.drop(['bairro'], axis=1)

In [3]:
list(df_novo)

['bairro_group',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimo_noites',
 'numero_de_reviews',
 'reviews_por_mes',
 'calculado_host_listings_count',
 'disponibilidade_365']

In [4]:
# Implementando On hot Encoder para transformação das variaveis categóricas

# Selecionar colunas categóricas
categorical_columns = df_novo.select_dtypes(include=['object']).columns.tolist()

# Aplicar One-Hot Encoding
encoder = OneHotEncoder(sparse=False)
one_hot_encoded = encoder.fit_transform(df_novo[categorical_columns])

# Criar um DataFrame com os dados codificados
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Garantir que os índices sejam os mesmos antes de concatenar
one_hot_df.index = df_novo.index

# Remover as colunas categóricas originais antes de concatenar
df_novo = df_novo.drop(columns=categorical_columns)

# Concatenar DataFrames
df_encoded = pd.concat([df_novo, one_hot_df], axis=1)

# Exibir o DataFrame final
df_encoded

,latitude,longitude,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365,bairro_group_Bronx,bairro_group_Brooklyn,bairro_group_Manhattan,bairro_group_Queens,bairro_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,40.75362,-73.98377,225,1,45,0,2,355,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,40.80902,-73.94190,150,3,0,0,1,365,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,40.68514,-73.95976,89,1,270,4,1,194,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,40.79851,-73.94399,80,10,9,0,1,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,40.74767,-73.97500,200,3,74,0,1,129,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48889,40.67853,-73.94995,70,2,0,0,2,9,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
48890,40.70184,-73.93317,40,4,0,0,2,36,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
48891,40.81475,-73.94867,115,10,0,0,1,27,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
48892,40.75751,-73.99112,55,1,0,0,6,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [5]:
# Normalizando os dados e separando em variáveis dependetes e variáveis independentes.
y = df_encoded["price"]
X = df_encoded[[ 'minimo_noites', 
                'disponibilidade_365', 
                'numero_de_reviews',
                'reviews_por_mes',
                'calculado_host_listings_count',
                'latitude',
                'longitude',
                'bairro_group_Bronx',
                'bairro_group_Brooklyn',
                'bairro_group_Manhattan',
                'bairro_group_Queens',
                'bairro_group_Staten Island',
                'room_type_Entire home/apt',
                'room_type_Private room',
                'room_type_Shared room'
                ]]

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [6]:
#Separando em treino e tste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Rede neural 
from sklearn.neural_network import MLPRegressor

# Treinando o modelo da Rede Neural
regr = MLPRegressor(hidden_layer_sizes=(5,5), 
                    random_state=1, 
                    max_iter=2000, 
                    tol=0.1)
regr.fit(X_train, y_train)

# Fazendo previsões
y_pred = regr.predict(X_test)

# Avaliando o modelo
mse_rede = mean_squared_error(y_test, y_pred)
rmse_rede = np.sqrt(mse_rede)
r2_rede = r2_score(y_test, y_pred)
mae_rede = mean_absolute_error(y_test, y_pred)

print(f"Rede Neural RMSE no conjunto de teste: {rmse_rede:.4f}")
print(f"Rede Neural R² no conjunto de teste: {r2_rede:.4f}")
print(f"Rede Neural MAE no conjunto de teste: {mae_rede:.4f}")

Rede Neural RMSE no conjunto de teste: 46.8412
Rede Neural R² no conjunto de teste: 0.5269
Rede Neural MAE no conjunto de teste: 33.8991


In [8]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

# Treinando o modelo da Random Forest
regr = RandomForestRegressor(min_samples_leaf=1,
           n_estimators= 1000,
           n_jobs=6,
           max_depth = 10,
           min_samples_split= 10,
           random_state=0)
model_random = regr.fit(X_train, y_train)

# Efetuando previsão
pred_random = model_random.predict(X_test)

# Avaliando o modelo
mse_random = mean_squared_error(y_test, pred_random)
rmse_random = np.sqrt(mse_random)
r2_random = r2_score(y_test, pred_random)
mae_random = mean_absolute_error(y_test, pred_random)

print(f"Random Forest RMSE no conjunto de teste: {rmse_random:.4f}")
print(f"Random Forest R² no conjunto de teste: {r2_random:.4f}")
print(f"Random Forest MAE no conjunto de teste: {mae_random:.4f}")

Random Forest RMSE no conjunto de teste: 44.9080
Random Forest R² no conjunto de teste: 0.5651
Random Forest MAE no conjunto de teste: 32.0751


In [9]:
# Gradient Boost
from sklearn.ensemble import GradientBoostingRegressor

#Treinando modelo Gradient Boost
reg = GradientBoostingRegressor(learning_rate=0.05,
    max_depth=3,
    n_estimators=1000)
model_boost = reg.fit(X_train, y_train)

# Efetuando Previsão
pred_boos = model_boost.predict(X_test)

# Avaliando o modelo
mse_boost= mean_squared_error(y_test, pred_boos)
rmse_boost = np.sqrt(mse_boost)
r2_boost = r2_score(y_test, pred_boos)
mae_boost = mean_absolute_error(y_test, pred_boos)

print(f"Gradient Boosting RMSE no conjunto de teste: {rmse_boost:.4f}")
print(f"Gradient Boosting R² no conjunto de teste: {r2_boost:.4f}")
print(f"Gradient Boosting MAE no conjunto de teste: {mae_boost:.4f}")

Gradient Boosting RMSE no conjunto de teste: 44.9447
Gradient Boosting R² no conjunto de teste: 0.5644
Gradient Boosting MAE no conjunto de teste: 32.2061


### Resposta pergunta 3


A previsão do preço foi realizada como um problema de regressão, uma vez que o objetivo é estimar um valor contínuo para o aluguel de imóveis com base em suas características.


Para garantir que os dados estivessem no formato adequado para o modelo, foram realizadas os seguintes passos. As variáveis **['id', 'nome', 'host_id', 'host_name', 'ultima_review', 'bairro']** foram excluídas, pois não contribuem para a predição do preço.
As variáveis **bairro_group** e **room_type** foram transformadas utilizando One-Hot Encoding, garantindo que as informações categóricas fossem representadas numericamente.
Foi implementado o StandardScaler para normalizar as variáveis numéricas e evitar que características com diferentes escalas impactassem de forma desproporcional o modelo.

#### Modelos testados
Foram avaliados três algoritmos de aprendizado de máquina:

- MLP Regressor (Rede Neural Multicamadas)
Prós: Capacidade de capturar padrões complexos nos dados.
Contras: Alto tempo de treinamento e sensibilidade à normalização dos dados.
- Gradient Boosting Regressor
Prós: Excelente capacidade preditiva e robustez contra overfitting.
Contras: Alto custo computacional e tuning de hiperparâmetros mais complexo.
- Random Forest Regressor (melhor desempenho)
Prós: Robustez a outliers, baixa necessidade de ajuste fino de hiperparâmetros e boa generalização.
Contras: Maior consumo de memória e interpretabilidade limitada em comparação a modelos lineares.

O Random Forest Regressor apresentou o melhor desempenho nos testes, com os seguintes resultados no conjunto de teste:

RMSE (Root Mean Squared Error): 44.9080 -> Indica que, em média, os erros estão na faixa de 44 dólares.\
R² (Coeficiente de Determinação): 0.5651 -> Indica que o modelo consegue explicar aproximadamente 56,5% da variação dos preços.\
MAE (Mean Absolute Error): 32.0751 -> Em média, a previsão do preço do aluguel apresenta um erro de aproximadamente 32 dólares.\

A métrica RMSE foi escolhida como principal critério de avaliação, pois penaliza erros grandes de maneira mais significativa, sendo adequada para cenários onde valores extremos podem impactar a previsão final. Além disso, o R² foi analisado para medir a proporção da variabilidade do preço explicada pelo modelo. Por fim, o MAE mede o erro absoluto médio das previsões.

Dessa forma, o modelo Random Forest foi considerado a melhor opção, equilibrando desempenho preditivo e robustez para o problema de precificação de aluguéis.

In [10]:
# Criar um DataFrame com as características do novo apartamento
novo_apartamento = pd.DataFrame([{
    'id': 2595,
    'nome': 'Skylit Midtown Castle',
    'host_id': 2845,
    'host_name': 'Jennifer',
    'bairro_group': 'Manhattan',
    'bairro': 'Midtown',
    'latitude': 40.75362,
    'longitude': -73.98377,
    'room_type': 'Entire home/apt',
    'minimo_noites': 1,
    'numero_de_reviews': 45,
    'ultima_review': '2019-05-21',
    'reviews_por_mes': 0.38,
    'calculado_host_listings_count': 2,
    'disponibilidade_365': 355
}])

In [11]:
                                  
novo_apartamento = novo_apartamento.drop(columns=['id', 'nome', 'host_id', 'host_name', 'ultima_review', 'bairro'])
categorical_columns = ['bairro_group', 'room_type']

# Aplicar One-Hot Encoding usando o encoder treinado
one_hot_encoded = encoder.transform(novo_apartamento[categorical_columns])

# Criar um DataFrame com os dados codificados
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Garantir que os índices sejam os mesmos antes de concatenar
one_hot_df.index = novo_apartamento.index

# Remover as colunas categóricas originais antes de concatenar
novo_apartamento = novo_apartamento.drop(columns=categorical_columns)

# Concatenar os DataFrames
novo_apartamento = pd.concat([novo_apartamento, one_hot_df], axis=1)

In [12]:
#Reordenando vairáveis
novo_apartamento = novo_apartamento[['minimo_noites', 
                      'disponibilidade_365', 
                      'numero_de_reviews',
                      'reviews_por_mes',
                      'calculado_host_listings_count',
                      'latitude',
                      'longitude',
                      'bairro_group_Bronx',
                      'bairro_group_Brooklyn',
                      'bairro_group_Manhattan',
                      'bairro_group_Queens',
                      'bairro_group_Staten Island',
                      'room_type_Entire home/apt',
                      'room_type_Private room',
                      'room_type_Shared room']]

In [13]:
# Normalizando 
novo_apartamento = scaler.transform(novo_apartamento)

In [14]:
#Previsão do novo apartamento
preco_previsto = model_random.predict(novo_apartamento)
print(f"Preço previsto para o apartamento: ${preco_previsto[0]:.2f}")

Preço previsto para o apartamento: $210.11


In [15]:
# Salvando modelo em um arquivo .pkl
import pickle

with open("modelo.pkl", "wb") as arquivo:
    pickle.dump(model_random, arquivo)